In [1]:
import pandas as pd
import numpy as np

In [2]:
import nltk

In [3]:
# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>


False

In [4]:
import re

In [5]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [6]:
# amazon=pd.read_csv("Eco_Friendly_Products_Test_Full.xlsx - Sheet1.csv")
amazon=pd.read_csv("Amazon Reviews Validation Data_1.csv")

In [7]:
amazon

,Reviews,Aspect
0,The settings are perfect for all hair types,Adaptability
1,with lots of settings,Adaptability
2,heating element has died,Durability
3,"Also, the diffuser does not stay on at all.",Ease of Use
4,and it won't stay on it at all. Just flys off....,Ease of Use
...,...,...
1176,top no longer stays on.,Performance
1177,unfortunately one of the lids does not fit at all,Performance
1178,"cheap enough that if my husband loses one, we ...",Price
1179,economical price,Price


In [8]:
# a=amazon["Review"]
a=amazon["Reviews"]

In [9]:
q=amazon["Aspect"]

In [10]:
all_values = q.str.split(',').explode()

In [11]:
unique_values = all_values.unique()

In [12]:
len_aspect=len(unique_values)

In [13]:
len_aspect

12

In [14]:
processed_series = a

In [15]:
processed_series

0             The settings are perfect for all hair types
1                                   with lots of settings
2                                heating element has died
3             Also, the diffuser does not stay on at all.
4       and it won't stay on it at all. Just flys off....
                              ...                        
1176                              top no longer stays on.
1177    unfortunately one of the lids does not fit at all
1178    cheap enough that if my husband loses one, we ...
1179                                     economical price
1180    Also, there is no way to seal the drinking hol...
Name: Reviews, Length: 1181, dtype: object

In [16]:
def extract_nouns(words):
    """Extract and return nouns from a given list of words."""
    # Join the list of words into a sentence
    # sentence = ' '.join(words)

    # Tokenize and POS tag the words in the sentence
    tokens = nltk.word_tokenize(words)
    parts_of_speech = nltk.pos_tag(tokens)

    # Extract and return words tagged as NN, NNS, NNP, or NNPS
    return [word for word, pos in parts_of_speech if pos in ['NN', 'NNS', 'NNP', 'NNPS']]


In [17]:
nouns = processed_series.apply(extract_nouns)
print(nouns)

0                                 [settings, hair, types]
1                                        [lots, settings]
2                                               [element]
3                                              [diffuser]
4                                                  [Just]
                              ...                        
1176                                              [stays]
1177                                               [lids]
1178                                            [husband]
1179                                              [price]
1180    [way, drinking, hole, car, bump, liquid, cup, ...
Name: Reviews, Length: 1181, dtype: object


In [18]:
from textblob import TextBlob

def extract_nouns_textblob(sentence):
    
    """Extract and return nouns from a given sentence using TextBlob."""
    # sentence = ' '.join(sentence)
    blob = TextBlob(sentence)
    return [word for word, tag in blob.tags if tag in ["NN", "NNS", "NNP", "NNPS"]]


In [19]:
nouns_testblob = processed_series.apply(extract_nouns_textblob)
print(nouns_testblob)

0                                 [settings, hair, types]
1                                        [lots, settings]
2                                               [element]
3                                              [diffuser]
4                                                      []
                              ...                        
1176                                              [stays]
1177                                               [lids]
1178                                            [husband]
1179                                              [price]
1180    [way, drinking, hole, car, bump, liquid, cup, ...
Name: Reviews, Length: 1181, dtype: object


In [20]:
import spacy

In [21]:
nlpt = spacy.load("en_core_web_sm")

In [22]:
def extract_nouns_spacy(sentence):
    """Extract and return nouns from a given sentence using spaCy."""
    # sentence = ' '.join(sentence)
    doc = nlpt(sentence)
    return [token.text for token in doc if token.pos_ in ["NOUN"]]

In [23]:
doc = processed_series.apply(extract_nouns_spacy)
doc

0                                 [settings, hair, types]
1                                        [lots, settings]
2                                      [heating, element]
3                                              [diffuser]
4                                                   [one]
                              ...                        
1176                                                [top]
1177                                               [lids]
1178                                            [husband]
1179                                              [price]
1180    [way, drinking, hole, car, bump, liquid, cup, ...
Name: Reviews, Length: 1181, dtype: object

In [24]:
doc[1180]

['way', 'drinking', 'hole', 'car', 'bump', 'liquid', 'cup', 'holder']

In [25]:
def load_glove_model(glove_file):
    """Load the GloVe model from a file."""
    print("Loading GloVe Model")
    with open(glove_file, 'r') as file:
        model = {}
        for line in file:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array([float(val) for val in split_line[1:]])
            model[word] = embedding
        print("Done. {} words loaded!".format(len(model)))
    return model

glove_model = load_glove_model("/Users/divyamsobti/Downloads/glove.6B/glove.6B.100d.txt")


Loading GloVe Model
Done. 400000 words loaded!


In [26]:
sentences_testblob=np.array(nouns_testblob)


In [27]:
sentences_spacy=np.array(doc)

In [28]:
sentences_nltk=np.array(nouns)

# TFidf


In [29]:
def extract_nouns_tf(sentence):

    # Tokenize and POS tag the words in the sentence
    tokens = nltk.word_tokenize(sentence)
    parts_of_speech = nltk.pos_tag(tokens)
    x=[word for word, pos in parts_of_speech if pos in ['NN', 'NNS', 'NNP', 'NNPS']]
    x=" ".join(x)
    # Extract and return words tagged as NN, NNS, NNP, or NNPS
    return x

In [30]:
nouns_tf = processed_series.apply(extract_nouns_tf)

In [31]:
nouns_tf

0                                settings hair types
1                                      lots settings
2                                            element
3                                           diffuser
4                                               Just
                            ...                     
1176                                           stays
1177                                            lids
1178                                         husband
1179                                           price
1180    way drinking hole car bump liquid cup holder
Name: Reviews, Length: 1181, dtype: object

In [32]:


def extract_nouns_textblob_tf(sentence):
    
    """Extract and return nouns from a given sentence using TextBlob."""
    blob = TextBlob(sentence)
    x=[word for word, tag in blob.tags if tag in ["NN", "NNS", "NNP", "NNPS"]]
    x=" ".join(x)
    return x


In [33]:
nouns_testblob_tf = processed_series.apply(extract_nouns_textblob_tf)
print(nouns_testblob_tf)

0                                settings hair types
1                                      lots settings
2                                            element
3                                           diffuser
4                                                   
                            ...                     
1176                                           stays
1177                                            lids
1178                                         husband
1179                                           price
1180    way drinking hole car bump liquid cup holder
Name: Reviews, Length: 1181, dtype: object


In [34]:
def extract_nouns_spacy_tf(sentence):
    """Extract and return nouns from a given sentence using spaCy."""
    doc = nlpt(sentence)
    x=[token.text for token in doc if token.pos_ in ["NOUN"]]
    x=" ".join(x)
    return x

In [35]:
doc_tf = processed_series.apply(extract_nouns_spacy_tf)
doc_tf

0                                settings hair types
1                                      lots settings
2                                    heating element
3                                           diffuser
4                                                one
                            ...                     
1176                                             top
1177                                            lids
1178                                         husband
1179                                           price
1180    way drinking hole car bump liquid cup holder
Name: Reviews, Length: 1181, dtype: object

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [37]:
# Step 2: Convert text to numerical vectors using TF-IDF
vectorizer = TfidfVectorizer()
nltk = vectorizer.fit_transform(nouns_tf).toarray()

In [38]:
spacy_=vectorizer.fit_transform(doc_tf).toarray()

In [39]:
TextBlob_=vectorizer.fit_transform(nouns_testblob_tf).toarray()

In [40]:
def sentence_vector(word_list, model, vector_size=100):
    """Generate a sentence vector by averaging the word vectors."""
    word_vectors = [model[word] for word in word_list if word in model]
    if len(word_vectors) == 0:
        return np.zeros(vector_size)
    return np.mean(word_vectors, axis=0)




In [41]:
sentence_vectors_testblob = np.array([sentence_vector(sentence, glove_model) for sentence in sentences_testblob])

In [42]:
sentence_vectors_spacy = np.array([sentence_vector(sentence, glove_model) for sentence in sentences_spacy])

In [43]:
sentence_vectors_nltk = np.array([sentence_vector(sentence, glove_model) for sentence in sentences_nltk])

# Cluster for 12 aspects

In [44]:
from sklearn.cluster import KMeans

num_clusters = len_aspect  # Example number of clusters

In [45]:
num_clusters = len_aspect  # Example number of clusters
kmeans_textblob = KMeans(n_clusters=num_clusters)
kmeans_textblob.fit(sentence_vectors_testblob)
labels_textblob = kmeans_textblob.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [46]:
kmeans_spacy = KMeans(n_clusters=num_clusters)
kmeans_spacy.fit(sentence_vectors_spacy)
labels_spacy = kmeans_spacy.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [47]:
kmeans_nltk = KMeans(n_clusters=num_clusters)
kmeans_nltk.fit(sentence_vectors_nltk)
labels_nltk = kmeans_nltk.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


# ploting

In [48]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"

In [49]:
from sklearn.manifold import TSNE


In [50]:
import plotly.graph_objects as go
import plotly.io as pio



## textblobl

In [51]:
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_testblob)

In [52]:


fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Textblob)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [53]:

# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_testblob)

# 3D Scatter plot
fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Textblob)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## spacy

In [54]:
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_spacy)

In [55]:
fig = go.Figure()


for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Spacy)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [56]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_spacy)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Spacy)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## NLTK


In [57]:
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_nltk)

In [58]:
fig = go.Figure()
for i in range(num_clusters):
    indices = labels_nltk == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(NLTK)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)

In [59]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_nltk)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_nltk == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(NLTK)',
                    scene=dict(xaxis_title='t-SNE Dimension 1',
                                 yaxis_title='t-SNE Dimension 2',
                                 zaxis_title='t-SNE Dimension 3'))
fig.show()


# kmeans with 6 aspects

In [60]:
num_clusters = 6 # Example number of clusters

In [61]:
kmeans_textblob = KMeans(n_clusters=num_clusters)
kmeans_textblob.fit(sentence_vectors_testblob)
labels_textblob = kmeans_textblob.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [62]:
kmeans_spacy = KMeans(n_clusters=num_clusters)
kmeans_spacy.fit(sentence_vectors_spacy)
labels_spacy = kmeans_spacy.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [63]:
kmeans_nltk = KMeans(n_clusters=num_clusters)
kmeans_nltk.fit(sentence_vectors_nltk)
labels_nltk = kmeans_nltk.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



# ploting

## textblobl

In [64]:
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_testblob)

In [65]:


fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Textblob)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [66]:

# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_testblob)

# 3D Scatter plot
fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Textblob)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## spacy

In [67]:
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_spacy)

In [68]:
fig = go.Figure()


for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Spacy)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [69]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_spacy)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Spacy)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## NLTK


In [70]:
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_nltk)

In [71]:
fig = go.Figure()
for i in range(num_clusters):
    indices = labels_nltk == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(NLTK)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)

In [72]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_nltk)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_nltk == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(NLTK)',
                    scene=dict(xaxis_title='t-SNE Dimension 1',
                                 yaxis_title='t-SNE Dimension 2',
                                 zaxis_title='t-SNE Dimension 3'))
fig.show()


# GMM

In [73]:
from sklearn.mixture import GaussianMixture

In [74]:
gmm = GaussianMixture(n_components = 12)

In [75]:
gmm_6 = GaussianMixture(n_components = 6)

In [76]:
gmm.fit(sentence_vectors_nltk)
labels_nltk = gmm.predict(sentence_vectors_nltk)

In [77]:
gmm.fit(sentence_vectors_spacy)
labels_spacy = gmm.predict(sentence_vectors_spacy)

In [78]:
gmm.fit(sentence_vectors_testblob)
labels_textblob = gmm.predict(sentence_vectors_testblob)

# ploting

In [79]:
num_clusters=12

## textblobl

In [80]:
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_testblob)

In [81]:


fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Textblob)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [82]:

# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_testblob)

# 3D Scatter plot
fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Textblob)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## spacy

In [83]:
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_spacy)

In [84]:
fig = go.Figure()


for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Spacy)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [85]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_spacy)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Spacy)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## NLTK


In [86]:
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_nltk)

In [87]:
fig = go.Figure()
for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(NLTK)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)

In [88]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_nltk)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(NLTK)',
                    scene=dict(xaxis_title='t-SNE Dimension 1',
                                 yaxis_title='t-SNE Dimension 2',
                                 zaxis_title='t-SNE Dimension 3'))
fig.show()


# 6 cluster

In [89]:
gmm_6.fit(sentence_vectors_nltk)
labels_nltk = gmm_6.predict(sentence_vectors_nltk)

In [90]:
gmm_6.fit(sentence_vectors_spacy)
labels_spacy = gmm_6.predict(sentence_vectors_spacy)

In [91]:
gmm_6.fit(sentence_vectors_testblob )
labels_textblob = gmm_6.predict(sentence_vectors_testblob)

In [92]:
num_clusters=6

## textblobl

In [93]:
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_testblob)

In [94]:


fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Textblob)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [95]:

# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_testblob)

# 3D Scatter plot
fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Textblob)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## spacy

In [96]:
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_spacy)

In [97]:
fig = go.Figure()


for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Spacy)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [98]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_spacy)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Spacy)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## NLTK


In [99]:
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_nltk)

In [100]:
fig = go.Figure()
for i in range(num_clusters):
    indices = labels_nltk == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(NLTK)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)

In [101]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_nltk)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_nltk == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(NLTK)',
                    scene=dict(xaxis_title='t-SNE Dimension 1',
                                 yaxis_title='t-SNE Dimension 2',
                                 zaxis_title='t-SNE Dimension 3'))
fig.show()


In [102]:
# Create DataFrame
df_spacy = pd.DataFrame({
    'Sentence': doc,
    'Cluster': labels_spacy
})

# Show the DataFrame
print(df_spacy)

                                               Sentence  Cluster
0                               [settings, hair, types]        4
1                                      [lots, settings]        0
2                                    [heating, element]        4
3                                            [diffuser]        3
4                                                 [one]        2
...                                                 ...      ...
1176                                              [top]        2
1177                                             [lids]        1
1178                                          [husband]        2
1179                                            [price]        5
1180  [way, drinking, hole, car, bump, liquid, cup, ...        4

[1181 rows x 2 columns]


In [103]:
# Create DataFrame
df_textblob = pd.DataFrame({
    'Sentence': sentences_testblob,
    'Cluster': labels_textblob
})

# Show the DataFrame
print(df_textblob)

                                               Sentence  Cluster
0                               [settings, hair, types]        4
1                                      [lots, settings]        4
2                                             [element]        1
3                                            [diffuser]        2
4                                                    []        2
...                                                 ...      ...
1176                                            [stays]        4
1177                                             [lids]        3
1178                                          [husband]        4
1179                                            [price]        0
1180  [way, drinking, hole, car, bump, liquid, cup, ...        4

[1181 rows x 2 columns]


In [104]:
# Create DataFrame
df_nltk = pd.DataFrame({
    'Sentence': sentences_nltk,
    'Cluster': labels_nltk
})

# Show the DataFrame
print(df_nltk)

                                               Sentence  Cluster
0                               [settings, hair, types]        4
1                                      [lots, settings]        1
2                                             [element]        1
3                                            [diffuser]        3
4                                                [Just]        3
...                                                 ...      ...
1176                                            [stays]        1
1177                                             [lids]        0
1178                                          [husband]        2
1179                                            [price]        2
1180  [way, drinking, hole, car, bump, liquid, cup, ...        1

[1181 rows x 2 columns]
